In [2]:
from bastionlab import Identity

# Create `Identity` for Data owner.
data_owner = Identity.create("data_owner")

In [1]:
!wget 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'

--2022-12-14 10:08:03--  https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60302 (59K) [text/plain]
Saving to: ‘titanic.csv’

titanic.csv         100%[===================>]  58.89K  --.-KB/s    in 0.01s   

2022-12-14 10:08:03 (4.46 MB/s) - ‘titanic.csv’ saved [60302/60302]



In [17]:
import polars as pl

df = pl.read_csv("titanic.csv")

from bastionlab import Connection

connection = Connection("localhost")

from bastionlab.polars.policy import Policy, Aggregation, Log

policy = Policy(safe_zone=Aggregation(min_agg_size=10), unsafe_handling=Log())
rdf = connection.client.polars.send_df(df, policy=policy, sanitized_columns=["Name"])

In [4]:
rdf.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [18]:
_rdf = rdf.to_dataset(
    ["Name", "Survived"],
    "Age",
    "distilbert-base-uncased",
    "distilbert-base-uncased",
)

# _rdf.fetch()

In [19]:
remote_datasets = connection.client.torch.list_remote_datasets()

In [8]:
! pip install transformers

In [20]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
from bastionlab.torch.utils import MultipleOutputWrapper

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False,
    torchscript=True,
)
model = MultipleOutputWrapper(model, 0)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier

In [21]:
from bastionlab.torch.optimizer_config import Adam

client = connection.client.torch
remote_learner = client.RemoteLearner(
    model,
    remote_datasets[0],
    max_batch_size=2,
    loss="cross_entropy",
    optimizer=Adam(lr=5e-5),
    model_name="DistilBERT",
)

ValueError: You have to specify either input_ids or inputs_embeds